In [1]:
import os
import glob
import jieba
import jieba.posseg as pseg
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import re

# 从多个停用词文件中加载停用词并合并
def load_stopwords_from_files(file_paths):
    stopwords = set()
    for path in file_paths:
        if os.path.exists(path):
            with open(path, 'r', encoding='utf-8') as f:
                stopwords.update([line.strip() for line in f if line.strip()])
        else:
            print(f"警告: 停用词文件 {path} 不存在")
    return stopwords

# 定义停用词文件列表（可根据需要扩展路径）
STOPWORDS_FILES = [
    '/home/mw/project/stopwords/停用词.txt',
    '/home/mw/project/stopwords/baidu_stopwords.txt',
    '/home/mw/project/stopwords/cn_stopwords.txt',
    '/home/mw/project/stopwords/hit_stopwords.txt',
    '/home/mw/project/stopwords/scu_stopwords.txt'
]

STOPWORDS = load_stopwords_from_files(STOPWORDS_FILES)

# 加载指定文件夹下所有txt文档
def load_documents(folder_path, encoding='utf-8'):
    documents = []
    file_pattern = os.path.join(folder_path, '*.txt')
    for file_path in glob.glob(file_pattern):
        with open(file_path, 'r', encoding=encoding) as f:
            documents.append(f.read())
    return documents

# 1. 数据清洗与预处理
def text_cleaning(text):
    # 去除特殊符号和标点
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9]', ' ', text)
    # 分词与词性标注
    words = pseg.cut(text)
    # 只保留名词、动词、形容词等实词且不在停用词列表中的词
    filtered_words = [word.word for word in words if word.flag in ['n', 'v', 'a'] and word.word not in STOPWORDS]
    return ' '.join(filtered_words)

# 2. 主题模型分析（LDA）
def lda_analysis(texts, n_topics=5):
    min_df_val = 1 if len(texts) < 2 else 2
    tf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=min_df_val, max_features=1000)
    tf = tf_vectorizer.fit_transform(texts)
    lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
    lda.fit(tf)
    feature_names = tf_vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(lda.components_):
        top_words = [feature_names[i] for i in topic.argsort()[:-11:-1]]
        print(f"主题 {topic_idx + 1}: {' '.join(top_words)}")
    return lda, tf_vectorizer

# 3. TF-IDF关键词提取（提取更多关键词并保留权重）
def tfidf_keywords(texts, top_n=20):
    tfidf = TfidfVectorizer(max_features=1000)
    tfidf_matrix = tfidf.fit_transform(texts)
    feature_names = tfidf.get_feature_names_out()
    doc_scores = tfidf_matrix[0].toarray().flatten()
    sorted_indices = np.argsort(doc_scores)[::-1]
    keywords = [(feature_names[i], doc_scores[i]) for i in sorted_indices[:top_n]]
    return keywords

# 4. 情感分析（单文档情感分析）
def sentiment_analysis(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity  # 返回情感极性(-1到1)

mask_path = '/home/mw/project/mask/mask_computer.png'
# 5. 根据 TF-IDF 权重生成词云（指定中文字体解决乱码问题）
def generate_wordcloud_from_frequencies(frequencies, output_path='future_industry_wordcloud.png', mask_path=None, width=800, height=600):
    if mask_path:
        mask = np.array(Image.open(mask_path))
    else:
        mask = None
    wc = WordCloud(
        background_color='white',
        max_words=500,
        contour_width=3,
        contour_color='steelblue',
        font_path='/home/mw/project/font/仿宋_GB2312.ttf',  # 请确保该字体文件存在或使用绝对路径
        width=width,
        height=height,
        mask=mask
    )
    wc.generate_from_frequencies(frequencies)
    plt.figure(figsize=(12, 8))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.show()
    wc.to_file(output_path)

if __name__ == "__main__":
    # 加载多个文档，替换为实际存放txt文件的文件夹路径
    folder_path = '/home/mw/project/document'
    docs = load_documents(folder_path)
    print(f"共加载 {len(docs)} 个文档")

    # 对每个文档进行清洗
    cleaned_docs = [text_cleaning(doc) for doc in docs]

    # 主题模型分析：传入多个文档
    lda, tf_vectorizer = lda_analysis(cleaned_docs)

    # TF-IDF关键词提取（此处以第一个文档为示例）
    keywords = tfidf_keywords(cleaned_docs)
    print("\nTF-IDF关键词及权重：")
    for kw, score in keywords:
        print(f"{kw}: {score:.4f}")

    # 情感分析（仅对第一个文档）
    polarity = sentiment_analysis(cleaned_docs[0])
    print(f"\n第1个文档情感极性: {polarity}")

    # 构造综合的TF-IDF权重词典（合并多个文档）
    tfidf = TfidfVectorizer(max_features=1000)
    tfidf_matrix = tfidf.fit_transform(cleaned_docs)
    feature_names = tfidf.get_feature_names_out()
    weights = np.sum(tfidf_matrix.toarray(), axis=0)
    freq_dict = {feature_names[i]: weights[i] for i in range(len(feature_names))}
    generate_wordcloud_from_frequencies(freq_dict)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


共加载 5 个文档


Loading model cost 0.792 seconds.
Prefix dict has been built successfully.


主题 1: 提供 应对 远程 模式 社会 增加 用于 医疗 方向 关系
主题 2: 创新 产业 交通 装备 领域 体系 构建 提升 强化 基础
主题 3: 科学技术 文化 社会 方式 结构 人工智能 创新 深远 传统 效率
主题 4: 道路 车辆 示范 驾驶 相关 汽车 主管部门 政府 行驶 智能网
主题 5: 提供 应对 远程 模式 社会 增加 用于 医疗 方向 关系

TF-IDF关键词及权重：
产业: 0.4975
创新: 0.2019
技术: 0.1951
企业: 0.1714
加快: 0.1646
场景: 0.1514
产品: 0.1457
装备: 0.1371
领域: 0.1298
重点: 0.1286
打造: 0.1286
智能: 0.1281
量子: 0.1280
推广: 0.1136
构建: 0.1114
推动: 0.1098
标准: 0.1081
推进: 0.1028
制造: 0.1009
体系: 0.0943

第1个文档情感极性: 0.0


<Figure size 1200x800 with 1 Axes>

一、紧扣 “创新驱动发展战略”，强化核心技术突破  
创新主体明确：TF-IDF 中 “企业”（0.1714）权重突出，体现企业作为创新主体的地位，契合国家 “强化企业创新主体地位” 的战略部署。词云高频词 “创新”“技术” 直接呼应创新驱动发展战略，表明通过技术创新推动产业升级是核心方向。  
前沿技术布局：“量子”“人工智能” 等词反映对前沿科技的关注，与国家 “瞄准量子信息、人工智能等战略性新兴产业” 的布局一致，通过突破关键核心技术，夯实创新驱动的科技基础。  
二、对接 “制造强国” 战略，推动产业高端化发展  
产业体系构建：主题 2 中 “产业”“装备”“制造”“体系”“构建” 等词，结合 TF-IDF 里 “产业”（0.4975）、“装备”（0.1371）等关键词，体现对高端装备制造、现代化产业体系建设的重视，契合制造强国战略中 “推动制造业高端化、智能化、绿色化发展” 的要求。  
新质生产力培育：词云 “智能”“技术” 与 “制造” 的关联，以及 TF-IDF 中 “智能”（0.1281）“推动”（0.1098）等词，指向以技术创新赋能制造业，通过智能化改造、技术转化应用，培育 “以科技创新为核心驱动力” 的新质生产力，实现从传统制造向先进制造的跃升。  
三、践行新发展理念，以创新引领协调发展  
创新理念贯穿全程：“创新” 一词在词云、主题、TF-IDF 中高频出现（如 “创新” TF-IDF 权重 0.2019），深度落实新发展理念中 “创新是引领发展的第一动力” 的要求，通过技术创新、产业创新打破传统发展路径依赖。  
产业协同发展导向：主题 2 中 “领域”“提升”“强化”“基础” 等词，体现产业发展中注重基础能力建设与多领域协同，呼应新发展理念中 “协调” 内涵，推动产业链上下游、科技与产业间的协调发展，构建更具竞争力的产业生态。  
四、锚定新质生产力，推动科技 - 产业深度融合  
科技成果转化应用：TF-IDF 中 “推广”（0.1136）“标准”（0.1081）等词，表明注重将 “量子”“人工智能” 等科技成果转化为产业标准与应用场景，符合新质生产力 “推动科技成果向现实生产力转化” 的要求。  
场景驱动产业升级：词云隐含的 “应用”“场景”（TF-IDF 中 “场景” 0.1514），指向通过智能网联汽车、医疗等应用场景拓展，加速技术落地，以场景驱动产业变革，培育经济发展新动能，正是新质生产力 “以新产业、新业态、新模式重塑经济结构” 的实践体现。

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 注册自定义字体
font_path = "/home/mw/project/font/仿宋_GB2312.ttf"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()

# 假设已经有了 TF-IDF 矩阵和特征名称
# tfidf_matrix 和 feature_names 是之前代码中已经计算得到的

# 1. 构建共现矩阵
co_occurrence_matrix = (tfidf_matrix.T * tfidf_matrix)
co_occurrence_matrix.setdiag(0)  # 将对角线元素置为 0，排除自身共现

# 2. 创建网络
G = nx.Graph()
for i in range(len(feature_names)):
    for j in range(i + 1, len(feature_names)):
        if co_occurrence_matrix[i, j] > 0:
            G.add_edge(feature_names[i], feature_names[j], weight=co_occurrence_matrix[i, j])

# 3. 分析网络拓扑结构
# 计算节点度
degree = dict(G.degree(weight='weight'))
# 计算度中心性
degree_centrality = nx.degree_centrality(G)
# 计算介数中心性
betweenness_centrality = nx.betweenness_centrality(G)

# 输出前 10 个度中心性最高的关键词
print("度中心性最高的前 10 个关键词：")
sorted_degree_centrality = sorted(degree_centrality.items(), key=lambda item: item[1], reverse=True)[:10]
for keyword, centrality in sorted_degree_centrality:
    print(f"{keyword}: {centrality:.4f}")

# 输出前 10 个介数中心性最高的关键词
print("\n介数中心性最高的前 10 个关键词：")
sorted_betweenness_centrality = sorted(betweenness_centrality.items(), key=lambda item: item[1], reverse=True)[:10]
for keyword, centrality in sorted_betweenness_centrality:
    print(f"{keyword}: {centrality:.4f}")

# 4. 可视化网络
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G, pos, node_size=[v * 1000 for v in degree.values()], node_color='lightblue')
nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.5)
# 此处直接调用，不传入 font_properties 参数，使用全局字体设置即可
nx.draw_networkx_labels(G, pos, font_size=10)
edge_labels = {(u, v): d['weight'] for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)
plt.title('关键词共现网络')
plt.axis('off')
plt.show()


度中心性最高的前 10 个关键词：
信息: 1.0000
先进: 1.0000
加快: 1.0000
国家: 1.0000
技术: 1.0000
控制: 1.0000
推动: 1.0000
支持: 1.0000
数据: 1.0000
智能: 1.0000

介数中心性最高的前 10 个关键词：
信息: 0.0042
先进: 0.0042
加快: 0.0042
国家: 0.0042
技术: 0.0042
控制: 0.0042
推动: 0.0042
支持: 0.0042
数据: 0.0042
智能: 0.0042


findfont: Font family 'FangSong_GB2312' not found.
findfont: Font family 'FangSong_GB2312' not found.
/opt/conda/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 20851 (\N{CJK UNIFIED IDEOGRAPH-5173}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 38190 (\N{CJK UNIFIED IDEOGRAPH-952E}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 35789 (\N{CJK UNIFIED IDEOGRAPH-8BCD}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 20849 (\N{CJK UNIFIED IDEOGRAPH-5171}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 29616 (\N{C

<Figure size 1200x800 with 1 Axes>

In [7]:
import networkx as nx
import matplotlib.pyplot as plt

# 注册自定义字体
font_path = "/home/mw/project/font/仿宋_GB2312.ttf"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()

# 假设已构建好 G、pos 等网络元素
plt.figure(figsize=(14, 10))

# 查询并设置正确字体名称（假设识别名称为 'FangSong'）
plt.rcParams['font.family'] = 'FangSong'

nx.draw_networkx_nodes(G, pos, node_size=[v * 2000 for v in degree.values()], node_color='lightblue')
nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.5)
nx.draw_networkx_labels(G, pos, font_size=12, font_weight='bold')
edge_labels = {(u, v): d['weight'] for u, v, d in G.edges(data=True)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)
plt.title('关键词共现网络', fontsize=16, fontweight='bold')
plt.axis('off')
plt.show()

findfont: Font family 'FangSong' not found.
findfont: Font family 'FangSong' not found.
/opt/conda/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 20851 (\N{CJK UNIFIED IDEOGRAPH-5173}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 38190 (\N{CJK UNIFIED IDEOGRAPH-952E}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 35789 (\N{CJK UNIFIED IDEOGRAPH-8BCD}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 20849 (\N{CJK UNIFIED IDEOGRAPH-5171}) missing from current font.
  fig.canvas.print_figure(bytes_io, **kw)
/opt/conda/lib/python3.11/site-packages/IPython/core/pylabtools.py:152: UserWarning: Glyph 29616 (\N{CJK UNIFIED IDE

<Figure size 1400x1000 with 1 Axes>